In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
# for some reason appending to PATH you need it to be string
sys.path.append(str(Path(cwd).parent / "src"))
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

Loaded environment variables from .env file.


In [2]:
import torch
from research_tools import get_gpus_available
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer


os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])
model_dtype = torch.bfloat16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device.type == "cuda", "No GPU available."

model_name = "meta-llama/Meta-Llama-3-8B"

model: LlamaForCausalLM = AutoModelForCausalLM.from_pretrained(
    model_name, token=hf_access_token, torch_dtype=model_dtype
)
model = model.to(device)

tokenizer: LlamaTokenizer = AutoTokenizer.from_pretrained(
    model_name, token=hf_access_token
)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
# from peft import get_peft_model, LoraConfig


# lora_rank = 64
# lora_alpha = 8

# lora_config = LoraConfig(
#     r=lora_rank,
#     lora_alpha=lora_alpha,
#     target_modules=["q_proj", "v_proj"],
# )

# model = get_peft_model(model, lora_config)

In [4]:
from unlearn_order.dataset import load_dataset

data_dir = Path("../data/random_bd")

splits = list(range(10))
n_train = 1
n_val = 1

train_files = [f"split_{splits[i]}.jsonl" for i in range(n_train)]
val_files = [f"split_{splits[i]}.jsonl" for i in range(n_train, n_train + n_val)]
combined_files = train_files + val_files

train_dataset = load_dataset(data_dir, train_files)
val_dataset = load_dataset(data_dir, val_files)
combined_dataset = load_dataset(data_dir, combined_files)

In [ ]:
from unlearn_order.pipeline import run_pipeline

batch_size = 4
tolerance = 0.01
lr = 3e-6
max_epochs = 100

run_pipeline(
    model,
    tokenizer,
    [
        ("f", "combined", combined_dataset),
        ("u", "unlearn", combined_dataset),
        ("e", "eval_train", train_dataset),
        ("e", "eval_val", val_dataset),
        ("f", "retrain_train", train_dataset),
        ("e", "eval_train", train_dataset),
        ("e", "eval_val", val_dataset),
    ],
    batch_size=batch_size,
    tolerance=tolerance,
    lr=lr,
    max_epochs=max_epochs,
)

  5%|▌         | 5/100 [01:56<39:20, 24.85s/it]

Epoch 5 loss: 0.067678130688561 acc: 0.3885350318471338


 10%|█         | 10/100 [03:52<37:13, 24.81s/it]

Epoch 10 loss: 0.024128436723391815 acc: 0.8248407643312102


 15%|█▌        | 15/100 [05:48<35:11, 24.85s/it]

...

 20%|██        | 20/100 [07:43<33:03, 24.80s/it]

Epoch 20 loss: 0.05382795949840242 acc: 0.3057324840764331


 25%|██▌       | 25/100 [09:39<31:01, 24.82s/it]

Epoch 25 loss: 0.055567541272397256 acc: 0.29936305732484075


 28%|██▊       | 28/100 [10:43<26:53, 22.41s/it]